In [16]:
import mysql.connector
import pprint

cnx = mysql.connector.connect(host='172.16.34.1', port='3307',
                              user='umls', password='umls', database='umls2022')
cur = cnx.cursor()

global counter
counter = 0

def find_cycle(given_cui):

    global counter
    # make sure only return the first 3 circles
    # once circles have  more than 40 different nodes, then return
    if counter > 3 or len(given_cui) > 40:
        return

    if len(given_cui) <= 1:
        # Exclude CUI1 = CUI2 with PAR relationships
        cur.execute("select CUI2 from MRREL where CUI1='%s' and REL='PAR' and CUI1!=CUI2" % given_cui[-1])
        query_result = cur.fetchall()
        for cui in query_result:
            # copy the current list and then add all CUI2 of given CUI1
            temp = given_cui.copy()
            temp.append(cui[0])
            # find circles
            find_cycle(temp)
        return

    else:
        # Exclude CUI1 = CUI2 with PAR relationships
        cur.execute("select CUI2 from MRREL where CUI1='%s' and REL='PAR' and CUI1!=CUI2 group by CUI2" % given_cui[-1])
        query_result = cur.fetchall()
        # if no CUI2 found in given CUI1, then return
        if len(query_result) < 1:
            return
        
        for cui in query_result:
            # if the current CUI2 is in the list, then we write the CUIs in a file
            if (cui[0] in given_cui):
                # make sure circles with at least 3 and 40 different CUIs
                if (len(given_cui)-given_cui.index(cui[0]))> 4 and len(given_cui) < 40:
                    # counter to help get the first 3 circles
                    counter += 1
                    # open .txt file 
                    file = open(f"{cui[0]}.txt", "w")
                    # write the squence of CUIS
                    for i in range(given_cui.index(cui[0]), len(given_cui)):
                        # use comma to separate them
                        file.write(str(given_cui[i]) + ',')
                    # add it again to print out a circle like required format   
                    file.write(str(cui[0]))
                    # file close
                    file.close()
                    # once we found a circle, then return
                    return
            else:
                #copy the current list and then add all CUI2 of given CUI1
                temp = given_cui.copy()
                temp.append(cui[0])
                # find circles
                find_cycle(temp)



if __name__ == "__main__":
    # gives a sequence of CUIs as starting points in the MRREL table
    cur.execute("""select CUI1, count(CUI1)
                from (select * from MRREL where REL = 'PAR' limit 1000000) as H
                where REL='PAR'
                group by CUI1
                order by count(CUI1) desc;""")
    test_query_result = cur.fetchall()
    for i in test_query_result[:10]:
        result = []
        # add the tested CUI1, and find the circle 
        result.append(i[0])
        find_cycle(result)